<a href="https://colab.research.google.com/github/Majdoddin/nlp/blob/develop/Pyannote_plays_and_Whisper_rhymes_v_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whisper's transcription plus Pyannote's Diarization 

Andrej Karpathy [suggested](https://twitter.com/karpathy/status/1574476200801538048?s=20&t=s5IMMXOYjBI6-91dib6w8g) training a classifier on top of  OpenAI [Whisper](https://openai.com/blog/whisper/) model features to identify the speaker, so we can visualize the speaker in the transcript. But, as [pointed out](https://twitter.com/tarantulae/status/1574493613362388992?s=20&t=s5IMMXOYjBI6-91dib6w8g) by Christian Perone, it seems that features from whisper wouldn't be that great for speaker recognition as its main objective is basically to ignore speaker differences.

In the following, I use [**`pyannote-audio`**](https://github.com/pyannote/pyannote-audio), a speaker diarization toolkit by Hervé Bredin, to identify the speakers, and then match it with the transcriptions of Whispr. I try it on a part of an [interview](https://youtu.be/NSp2fEQ6wyA) with Freeman Dyson. Check the result [**here**](https://majdoddin.github.io/dyson.html).

To make it easier to match the transcriptions to diarizations by speaker change, Sarah Kaiser [suggested](https://github.com/openai/whisper/discussions/264#discussioncomment-3825375) runnnig the pyannote.audio first and  then just running whisper on the split-by-speaker chunks. 
For sake of performance (and transcription quality?), we attach the audio segements into a single audio file with a silent spacer as a seperator, and run whisper on it. Enjoy it!

(For sake of performance , I also tried attaching the audio segements into a single audio file with a silent -or beep- spacer as a seperator, and run whisper on it see it on [colab](https://colab.research.google.com/drive/1HuvcY4tkTHPDzcwyVH77LCh_m8tP-Qet?usp=sharing). It [works](https://majdoddin.github.io/lexicap.html) on some audio, and fails on some (Dyson's Interview). The problem is, whisper does not reliably make a timestap on a spacer. See the discussions [#139](https://github.com/openai/whisper/discussions/139) and [#29](https://github.com/openai/whisper/discussions/29))

# Preparing the audio file

**Optional:** Mount Google Drive



In [1]:
from google.colab import drive
from pathlib import Path

drive_mount_path = Path("/content/drive")
drive.mount(str(drive_mount_path))
drive_mount_path /= "MyDrive"

Mounted at /content/drive


In [2]:
#@markdown Enter the URL of the YouTube video, or the path to the video/audio file you want to transcribe, give the output path, and run the cell. HTML file is generated only for YouTube videos

Source = 'Youtube' #@param ['Youtube', 'File (Google Drive)']
#@markdown ---
#@markdown #### **Youtube video**
video_url = "https://youtu.be/NSp2fEQ6wyA" #@param {type:"string"}
#store_audio = True #@param {type:"boolean"}
#@markdown ---
#@markdown #### **Google Drive video or audio (mp4, wav)**
video_path = "/content/drive/MyDrive/Colab Notebooks/PyannoteWhisper/dyson.mp4.webm" #@param {type:"string"}
#@markdown ---
output_path = "/content/drive/MyDrive/Colab Notebooks/PyannoteWhisper/" #@param {type:"string"}
output_path = str(Path(output_path))
#@markdown ---
#@markdown **Run this cell again if you change the video.**


In [3]:
Path(output_path).mkdir(parents=True, exist_ok=True)
%cd {output_path}
video_title = ""
video_id = ""

/content/drive/MyDrive/Colab Notebooks/PyannoteWhisper


## From YouTube

 Installing [`yt-dlp`](https://github.com/yt-dlp/yt-dlp) and downloading the [video](https://youtu.be/NSp2fEQ6wyA) from youtube.

In [4]:
if Source == "Youtube":
  !pip install -U yt-dlp

Custom build of `ffmpeg` as [recommended](https://github.com/yt-dlp/yt-dlp#strongly-recommended) by `yt-dlp`.

In [5]:
if Source == "Youtube":
  !wget -O - -q  https://github.com/yt-dlp/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz | xz -qdc| tar -x

In [6]:
#Getting video info
if Source == "Youtube":
  from yt_dlp import YoutubeDL
  with YoutubeDL() as ydl: 
    info_dict = ydl.extract_info(video_url, download=False)
    video_title = info_dict.get('title', None)
    video_id = info_dict.get('id', None)
    print("Title: " + video_title) # <= Here, you got the video title


Downloading the audio from YouTube.

In [7]:
if Source == "Youtube":
  !yt-dlp -xv --ffmpeg-location ffmpeg-master-latest-linux64-gpl/bin --audio-format wav  -o "{str(output_path) + '/'}input.wav" -- {video_url}

## or from File (Google Drive)

In [17]:

!



ffmpeg-master-latest-linux64-gpl  input_prep


In [9]:
if Source == 'File (Google Drive)':
    !ffmpeg -i {repr(video_path)} -vn -acodec pcm_s16le -ar 16000 -ac 1 input.wav  

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

## Prepending a spacer

`pyannote.audio` seems to miss the first 0.5 seconds of the audio, and, therefore, we prepend a spcacer.

In [9]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from pydub import AudioSegment

spacermilli = 2000
spacer = AudioSegment.silent(duration=spacermilli)


audio = AudioSegment.from_wav("input.wav") 

audio = spacer.append(audio, crossfade=0)

audio.export('input_prep.wav', format='wav')

<_io.BufferedRandom name='input_prep'>

# Pyannote's Diarization

[`pyannote.audio`](https://github.com/pyannote/pyannote-audio) is an open-source toolkit written in Python for **speaker diarization**. 

Based on [`PyTorch`](https://pytorch.org) machine learning framework, it provides a set of trainable end-to-end neural building blocks that can be combined and jointly optimized to build speaker diarization pipelines. 

`pyannote.audio` also comes with pretrained [models](https://huggingface.co/models?other=pyannote-audio-model) and [pipelines](https://huggingface.co/models?other=pyannote-audio-pipeline) covering a wide range of domains for voice activity detection, speaker segmentation, overlapped speech detection, speaker embedding reaching state-of-the-art performance for most of them. 

Installing `pyannote.audio`.

In [11]:
!pip install   pyannote.audio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 390 kB 5.1 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |████████████████████████████████| 585 kB 53.9 MB/s 
     |████████████████████████████████| 41 kB 480 kB/s 
     |████████████████████████████████| 498 kB 51.5 MB/s 
     |████████████████████████████████| 217 kB 47.8 MB/s 
     |████████████████████████████████| 111 kB 59.7 MB/s 
     |████████████████████████████████| 51 kB 197 kB/s 
     |████████████████████████████████| 163 kB 75.0 MB/s 
     |████████████████████████████████| 47 kB 5.7 MB/s 
     |████████████████████████████████| 529 kB 77.9 MB/s 
     |████████████████████████████████| 60 kB 8.0 MB/s 
     |████████████████████████████████| 117 kB 77.4 MB/s 
     |████████████████████████████████| 130 kB 75.4 MB/s 
     |████████████████████████████████| 348 kB 64.6 MB/s 
     |████████████████████████████████| 81

**Important:** To load the speaker diarization pipeline, 

* accept the user conditions on both [hf.co/pyannote/speaker-diarization](https://hf.co/pyannote/speaker-diarization) and [hf.co/pyannote/segmentation](https://huggingface.co/pyannote/segmentation).
* login using `notebook_login` below

In [12]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [13]:
from pyannote.audio import Pipeline

pipeline = Pipeline.from_pretrained('pyannote/speaker-diarization', use_auth_token=True)

Downloading:   0%|          | 0.00/500 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/318 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129k [00:00<?, ?B/s]

Running pyannote.audio to generate the diarizations.

In [14]:
DEMO_FILE = {'uri': 'blabla', 'audio': 'input_prep.wav'}
dz = pipeline(DEMO_FILE)  

with open("diarization.txt", "w") as text_file:
    text_file.write(str(dz))

In [15]:
print(*list(dz.itertracks(yield_label = True))[:10], sep="\n")

(<Segment(1.98281, 20.0391)>, 'F', 'SPEAKER_01')
(<Segment(20.8659, 34.1803)>, 'G', 'SPEAKER_01')
(<Segment(34.8553, 37.8759)>, 'H', 'SPEAKER_01')
(<Segment(38.8209, 40.4747)>, 'I', 'SPEAKER_01')
(<Segment(41.7741, 46.0772)>, 'J', 'SPEAKER_01')
(<Segment(46.9547, 51.6122)>, 'K', 'SPEAKER_01')
(<Segment(52.2534, 53.5528)>, 'L', 'SPEAKER_01')
(<Segment(55.2234, 56.3709)>, 'M', 'SPEAKER_01')
(<Segment(57.9741, 60.3703)>, 'N', 'SPEAKER_01')
(<Segment(61.2984, 70.4953)>, 'O', 'SPEAKER_01')


# Preparing audio files according to the diarization

In [16]:
def millisec(timeStr):
  spl = timeStr.split(":")
  s = (int)((int(spl[0]) * 60 * 60 + int(spl[1]) * 60 + float(spl[2]) )* 1000)
  return s

Grouping the diarization segments according to the speaker.

In [17]:
import re
dzs = open('diarization.txt').read().splitlines()

groups = []
g = []
lastend = 0

for d in dzs:   
  if g and (g[0].split()[-1] != d.split()[-1]):      #same speaker
    groups.append(g)
    g = []
  
  g.append(d)
  
  end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=d)[1]
  end = millisec(end)
  if (lastend > end):       #segment engulfed by a previous segment
    groups.append(g)
    g = [] 
  else:
    lastend = end
if g:
  groups.append(g)
print(*groups, sep='\n')

['[ 00:00:01.982 -->  00:00:20.039] F SPEAKER_01', '[ 00:00:20.865 -->  00:00:34.180] G SPEAKER_01', '[ 00:00:34.855 -->  00:00:37.875] H SPEAKER_01', '[ 00:00:38.820 -->  00:00:40.474] I SPEAKER_01', '[ 00:00:41.774 -->  00:00:46.077] J SPEAKER_01', '[ 00:00:46.954 -->  00:00:51.612] K SPEAKER_01', '[ 00:00:52.253 -->  00:00:53.552] L SPEAKER_01', '[ 00:00:55.223 -->  00:00:56.370] M SPEAKER_01', '[ 00:00:57.974 -->  00:01:00.370] N SPEAKER_01', '[ 00:01:01.298 -->  00:01:10.495] O SPEAKER_01', '[ 00:01:11.237 -->  00:01:15.692] P SPEAKER_01', '[ 00:01:16.536 -->  00:01:23.657] Q SPEAKER_01', '[ 00:01:24.265 -->  00:01:31.386] R SPEAKER_01', '[ 00:01:32.618 -->  00:01:33.192] S SPEAKER_01', '[ 00:01:36.060 -->  00:01:37.056] T SPEAKER_01', '[ 00:01:39.402 -->  00:01:40.819] U SPEAKER_01', '[ 00:01:42.490 -->  00:01:45.510] V SPEAKER_01', '[ 00:01:47.114 -->  00:01:53.172] W SPEAKER_01', '[ 00:01:54.539 -->  00:01:56.918] X SPEAKER_01']
['[ 00:01:57.947 -->  00:02:04.242] A SPEAKER_00'

Save the audio part corresponding to each diarization group.

In [18]:
audio = AudioSegment.from_wav("input_prep.wav")
gidx = -1
for g in groups:
  start = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
  end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[-1])[1]
  start = millisec(start) #- spacermilli
  end = millisec(end)  #- spacermilli
  print(start, end)
  gidx += 1
  audio[start:end].export(str(gidx) + '.wav', format='wav')

1982 116918
117947 124242
123719 224445
224445 233001
232697 239549
236241 237152
241928 248897
248509 258449


Freeing up some memory

In [ ]:
#del   DEMO_FILE, pipeline, spacer,  audio, dz, newAudio

# Whisper's Transcriptions

Installing Open AI whisper.

**Important:** There is a version conflict with pyannote.audio resulting in an error (see this [RP](https://github.com/pyannote/pyannote-audio/pull/1098)). Our workaround is to first run Pyannote and then whisper. You can safely ignore the error.

In [19]:
!pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-5td78po7
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-5td78po7
     |████████████████████████████████| 5.5 MB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 63.5 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175218 sha256=411b398951481188492ddb1432ca716102474bb7440f7f4a1b096c17768c39dd
  Stored in directory: /tmp/pip-ephem-wheel-cache-d5siuozd/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper


Run whisper on all audio files. Whisper generates the transcription and writes it to a file.

In [20]:
for i in range(gidx+1):
  !whisper {str(i) + '.wav'} --language en --model large

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
Moving 0 files to the new cache system
0it [00:00, ?it/s]
100%|█████████████████████████████████████| 2.87G/2.87G [00:32<00:00, 94.5MiB/s]
tcmalloc: large alloc 3087007744 bytes == 0xb074000 @  0x7fb5efae11e7 0x4b2590 0x5ad01c 0x5dcfef 0x58f92b 0x590c33 0x5e48ac 0x4d20fa 0x51041f 0x58fd37 0x50c4fc 0x5b4ee6 0x58ff2e 0x50d482 0x58fd37 0x50c4fc 0x5b4ee6 0x6005a3 0x607796 0x60785c 0x60a436 0x64db82 0x64dd2e 0x7fb5ef6dec87 0x5b636a
[00:00.000 --> 00:08.460]  So then I come to Cambridge in 1941 as a 17-year-old, and before, I mean, I'd always been interested
[00:08.460 --> 00:15.440]  in physics and applied mathematics of all sorts, and one of the textbooks that I bought
[00:15.440 --> 00:21.900]  as a prize was a textbook in aerodynamics, which I think was becaus

# Generating the HTML and/or txt file from the Transcriptions and the Diarization

Reading the transcription file.

In [21]:
!pip install -U webvtt-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Change or add to the speaker names and collors bellow as you wish `(speaker, textbox color, speaker color)`.

In [22]:
speakers = {'SPEAKER_00':('Dyson', 'white', 'darkorange'), 'SPEAKER_01':('Interviewer', '#e1ffc7', 'darkgreen') }
def_boxclr = 'white'
def_spkrclr = 'orange'

In the generated HTML,  the transcriptions for each diarization group are written in a box, with the speaker name on the top. By clicking a transcription, the embedded video jumps to the right time .

In [24]:
preS = '<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta http-equiv="X-UA-Compatible" content="ie=edge">\n    <title>' + \
      video_title + \
      '</title>\n    <style>\n        body {\n            font-family: sans-serif;\n            font-size: 18px;\n            color: #111;\n            padding: 0 0 1em 0;\n\t        background-color: #efe7dd;\n        }\n        table {\n             border-spacing: 10px;\n        }\n        th { text-align: left;}\n        .lt {\n          color: inherit;\n          text-decoration: inherit;\n        }\n        .l {\n          color: #050;\n        }\n        .s {\n            display: inline-block;\n        }\n        .c {\n            display: inline-block;\n        }\n        .e {\n            /*background-color: white; Changing background color */\n            border-radius: 20px; /* Making border radius */\n            width: fit-content; /* Making auto-sizable width */\n            height: fit-content; /* Making auto-sizable height */\n            padding: 5px 30px 5px 30px; /* Making space around letters */\n            font-size: 18px; /* Changing font size */\n            display: flex;\n            flex-direction: column;\n            margin-bottom: 10px;\n            white-space: nowrap;\n        }\n\n        .t {\n            display: inline-block;\n        }\n        #player {\n            position: sticky;\n            top: 20px;\n            float: right;\n        }\n    </style>\n\t<script>\n      var tag = document.createElement(\'script\');\n      tag.src = "https://www.youtube.com/iframe_api";\n      var firstScriptTag = document.getElementsByTagName(\'script\')[0];\n      firstScriptTag.parentNode.insertBefore(tag, firstScriptTag);\n      var player;\n      function onYouTubeIframeAPIReady() {\n        player = new YT.Player(\'player\', {\n          //height: \'210\',\n          //width: \'340\',\n          videoId: \'' + \
      video_id + \
      '\',\n        });\n      }\n      function jumptoTime(timepoint, id) {\n        event.preventDefault();\n        history.pushState(null, null, "#"+id);\n        player.seekTo(timepoint);\n        player.playVideo();\n      }\n    </script>\n  </head>\n  <body>\n    <h2>' + \
      video_title + \
      '</h2>\n  <i>Click on a part of the transcription, to jump to its video, and get an anchor to it in the address bar<br><br></i>\n<div  id="player"></div>\n'
postS = '\t</body>\n</html>'

In [34]:
import webvtt

from datetime import timedelta

html = list(preS)
txt = list("")
gidx = -1
for g in groups:  
  shift = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
  shift = millisec(shift) - spacermilli #the start time in the original video
  shift=max(shift, 0)
  
  gidx += 1
  captions = [[(int)(millisec(caption.start)), (int)(millisec(caption.end)),  caption.text] for caption in webvtt.read(str(gidx) + '.wav.vtt')]
  #captions = (list) webvtt.read(str(gidx) + '.wav.vtt')

  if captions:
    speaker = g[0].split()[-1]
    boxclr = def_boxclr
    spkrclr = def_spkrclr
    if speaker in speakers:
      speaker, boxclr, spkrclr = speakers[speaker] 
    
    html.append(f'<div class="e" style="background-color: {boxclr}">\n');
    html.append(f'<span style="color: {spkrclr}">{speaker}</span><br>\n')
      
    for c in captions:
      start = shift + c[0] 
      start = start / 1000.0   #time resolution ot youtube is Second.
      startStr = '{0:02d}:{1:02d}:{2:06.3f}'.format((int)(start // 3600), 
                                              (int)(start % 3600 // 60), 
                                              start % 60)
      end = shift + c[1] 
      end = end / 1000.0   #time resolution ot youtube is Second.
      endStr = '{0:02d}:{1:02d}:{2:06.3f}'.format((int)(end // 3600), 
                                              (int)(end % 3600 // 60), 
                                              end % 60)            
      
      txt.append(f'[{startStr} --> {endStr}] [{speaker}] {c[2]}\n')
      html.append(f'\t\t\t\t<a href="#{startStr}" id="{startStr}" class="lt" onclick="jumptoTime({int(start)}, this.id)">{c[2]}</a>\n')      

    html.append(f'</div>\n');

html.append(postS)

with open("capspeaker.txt", "w") as file:
  s = "".join(txt)
  file.write(s)
if Source == 'File (Google Drive)':
  print(s)
elif Source == 'Youtube':
  with open("capspeaker.html", "w") as file:    #TODO: proper embed tag for video/audio from file
    s = "".join(html)
    file.write(s)
    print(s)

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Freeman Dyson - Pure mathematics at Cambridge: the influence of Besicovitch (23/157)</title>
    <style>
        body {
            font-family: sans-serif;
            font-size: 18px;
            color: #111;
            padding: 0 0 1em 0;
	        background-color: #efe7dd;
        }
        table {
             border-spacing: 10px;
        }
        th { text-align: left;}
        .lt {
          color: inherit;
          text-decoration: inherit;
        }
        .l {
          color: #050;
        }
        .s {
            display: inline-block;
        }
        .c {
            display: inline-block;
        }
        .e {
            /*background-color: white; Changing background color */
            border-radius: 20px; /* Making border radius */
            widt

In [37]:
  !yt-dlp -v --ffmpeg-location ffmpeg-master-latest-linux64-gpl/bin   -o "dyson.mp4" -- {video_url}

[debug] Command-line config: ['-v', '--ffmpeg-location', 'ffmpeg-master-latest-linux64-gpl/bin', '-o', 'dyson.mp4', '--', 'https://youtu.be/NSp2fEQ6wyA']
[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out UTF-8, error UTF-8, screen UTF-8
[debug] yt-dlp version 2022.11.11 [8b64402] (pip)
[debug] Python 3.7.15 (CPython x86_64 64bit) - Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic (OpenSSL 1.1.1  11 Sep 2018, glibc 2.26)
[debug] exe versions: ffmpeg N-109048-g65f3bc9e7e-20221110 (setts), ffprobe N-109048-g65f3bc9e7e-20221110
[debug] Optional libraries: Cryptodome-3.15.0, brotli-1.0.9, certifi-2022.09.24, mutagen-1.46.0, sqlite3-2.6.0, websockets-10.4
[debug] Proxy map: {}
[debug] Loaded 1723 extractors
[debug] [youtube] Extracting URL: https://youtu.be/NSp2fEQ6wyA
[youtube] NSp2fEQ6wyA: Downloading webpage
[youtube] NSp2fEQ6wyA: Downloading android player API JSON
[debug] Sort order given by extractor: quality, res, fps, hdr:12, source, vcodec:vp9.2, channels, acodec, lang, prot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [20]:
  !pip install -U yt-dlp
  !yt-dlp -v --ffmpeg-location ffmpeg-master-latest-linux64-gpl/bin -o "dyson.mp4" -- {video_url}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 5.0 MB/s 
     |████████████████████████████████| 2.3 MB 51.7 MB/s 
     |████████████████████████████████| 193 kB 71.9 MB/s 
     |████████████████████████████████| 106 kB 73.7 MB/s 
     |████████████████████████████████| 357 kB 57.9 MB/s 
[debug] Command-line config: ['-v', '--ffmpeg-location', 'ffmpeg-master-latest-linux64-gpl/bin', '-o', 'dyson.mp4', '--', 'https://youtu.be/NSp2fEQ6wyA']
[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out UTF-8, error UTF-8, screen UTF-8
[debug] yt-dlp version 2022.11.11 [8b64402] (pip)
[debug] Python 3.7.15 (CPython x86_64 64bit) - Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic (OpenSSL 1.1.1  11 Sep 2018, glibc 2.26)
[debug] exe versions: none
[debug] Optional libraries: Cryptodome-3.15.0, brotli-1.0.9, certifi-2022.09.24, mutagen-1.46.0, sqlite3-2.6.0, websockets-10.4
[debug] Proxy map: {}
[de